### Convert VGG Via Annotation file into Yolo Format

In [1]:
import os
import argparse
import xml.etree.ElementTree as ET
def convert_vgg_to_yolo(vgg_folder, output_folder, class_file):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(class_file, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    for filename in os.listdir(vgg_folder):
        if not filename.endswith('.xml'):
            continue

        xml_path = os.path.join(vgg_folder, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        img_width = int(root.find('size/width').text)
        img_height = int(root.find('size/height').text)

        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_path = os.path.join(output_folder, txt_filename)

        with open(txt_path, 'w') as f:
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                if class_name not in classes:
                    continue

                class_id = classes.index(class_name)
                bbox = obj.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)

                x_center = (xmin + xmax) / 2 / img_width
                y_center = (ymin + ymax) / 2 / img_height
                w = (xmax - xmin) / img_width
                h = (ymax - ymin) / img_height

                f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

        print(f"Converted {filename} to YOLO format.")

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--vgg_folder', type=str, required=True,
                        help='Folder containing VGG annotation files')
    parser.add_argument('--output_folder', type=str, required=True,
                        help='Folder to save YOLO annotation files')
    parser.add_argument('--class_file', type=str, required=True,
                        help='File containing class names, one per line')
    args = parser.parse_args()

    convert_vgg_to_yolo(args.vgg_folder, args.output_folder, args.class_file)

### Convert VGG Via Annotation file into COCO Format

In [ ]:
import json


with open('path/to/vgg/annotation/file.json', 'r') as f:
    vgg_data = json.load(f)


coco_data = {
    "info": {
        "description": "My Dataset",
        "url": "",
        "version": "1.0",
        "year": 2023,
        "contributor": "",
        "date_created": "2023-03-21"
    },
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 1,
            "name": "class1",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "class2",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "class3",
            "supercategory": ""
        }
    ]
}


for image in vgg_data['images']:
    coco_image = {
        "id": image['id'],
        "file_name": image['filename'],
        "height": image['height'],
        "width": image['width']
    }
    coco_data['images'].append(coco_image)


for annotation in vgg_data['annotations']:
    coco_annotation = {
        "id": annotation['id'],
        "image_id": annotation['image_id'],
        "category_id": annotation['category_id'],
        "bbox": annotation['bbox'],
        "area": annotation['area'],
        "iscrowd": 0
    }
    coco_data['annotations'].append(coco_annotation)


with open('path/to/new/coco/annotation/file.json', 'w') as f:
    json.dump(coco_data, f)

### Convert VGG Via Annotation file into YOLO Format plus augment images using noise and changing brightness

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import random
from multiprocessing import Pool

def convert_annotation_file(vgg_file_path, yolo_file_path):
    with open(vgg_file_path, 'r') as f:
        lines = f.readlines()

    img_path = lines[0].strip()
    img = cv2.imread(img_path)
    height, width, channels = img.shape

    yolo_lines = []
    for line in lines[1:]:
        data = line.strip().split()
        class_name = data[0]
        x_min, y_min, x_max, y_max = list(map(float, data[1:]))

        x_center = (x_min + x_max) / 2 / width
        y_center = (y_min + y_max) / 2 / height
        box_width = (x_max - x_min) / width
        box_height = (y_max - y_min) / height

        yolo_lines.append(f"{class_name} {x_center} {y_center} {box_width} {box_height}\n")

    with open(yolo_file_path, 'w') as f:
        f.writelines(yolo_lines)


def augment_image(img_path, save_dir):
    img = Image.open(img_path)
    img_name, ext = os.path.splitext(os.path.basename(img_path))

    # Apply random brightness
    brightness_levels = [-70, -35, 0, 35, 70]
    for i, level in enumerate(brightness_levels):
        new_img = np.array(img)
        new_img = new_img + level
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        new_img.save(os.path.join(save_dir, f"{img_name}_brightness_{i+1}{ext}"))

    # Apply random noise
    noise_levels = [10, 20, 30, 40, 50]
    for i, level in enumerate(noise_levels):
        new_img = np.array(img)
        noise = np.random.normal(0, level, size=new_img.shape)
        new_img = new_img + noise
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        new_img.save(os.path.join(save_dir, f"{img_name}_noise_{i+1}{ext}"))


if __name__ == '__main__':
    # Set paths
    vgg_file_path = 'path/to/vgg/annotation/file.txt'
    yolo_file_path = 'path/to/yolo/annotation/file.txt'
    img_dir = 'path/to/image/directory'
    save_dir = 'path/to/save/augmented/images'
    
    
    convert_annotation_file(vgg_file_path, yolo_file_path)

    
    img_paths = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
    with Pool() as pool:
        pool.starmap(augment_image, [(img_path, save_dir) for img_path in img_paths])

### Convert VGG Via Annotation file into YOLO Format plus augment images for resizing

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import random
from multiprocessing import Pool

def convert_annotation_file(vgg_file_path, yolo_file_path, img_size):
    with open(vgg_file_path, 'r') as f:
        lines = f.readlines()

    img_path = lines[0].strip()
    img = cv2.imread(img_path)
    height, width, channels = img.shape

    yolo_lines = []
    for line in lines[1:]:
        data = line.strip().split()
        class_name = data[0]
        x_min, y_min, x_max, y_max = list(map(float, data[1:]))

        x_center = (x_min + x_max) / 2 / width
        y_center = (y_min + y_max) / 2 / height
        box_width = (x_max - x_min) / width
        box_height = (y_max - y_min) / height

        yolo_lines.append(f"{class_name} {x_center} {y_center} {box_width} {box_height}\n")

    with open(yolo_file_path, 'w') as f:
        f.writelines(yolo_lines)


def augment_image(img_path, save_dir, img_size):
    img = Image.open(img_path)
    img_name, ext = os.path.splitext(os.path.basename(img_path))

    
    brightness_levels = [-70, -35, 0, 35, 70]
    for i, level in enumerate(brightness_levels):
        new_img = np.array(img)
        new_img = new_img + level
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        
        
        new_img = new_img.resize((img_size, img_size))
        
        new_img.save(os.path.join(save_dir, f"{img_name}_brightness_{i+1}{ext}"))

    
    noise_levels = [10, 20, 30, 40, 50]
    for i, level in enumerate(noise_levels):
        new_img = np.array(img)
        noise = np.random.normal(0, level, size=new_img.shape)
        new_img = new_img + noise
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        
        # Resize image
        new_img = new_img.resize((img_size, img_size))
        
        new_img.save(os.path.join(save_dir, f"{img_name}_noise_{i+1}{ext}"))


if __name__ == '__main__':
    
    vgg_file_path = 'path/to/vgg/annotation/file.txt'
    yolo_file_path = 'path/to/yolo/annotation/file.txt'
    img_dir = 'path/to/image/directory'
    save_dir = 'path/to/save/augmented/images'
    img_size = 416  # 
    
    
    convert_annotation_file(vgg_file_path, yolo_file_path, img_size)

    
    img_paths = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
    with Pool() as pool:
        pool.starmap(augment_image, [(img_path, save_dir, img_size) for img_path in img_paths])